# 3. All about SPARQL

This module is about many of the parts of the SPARQL query language.

---

## 3.1. Finding out all about SPARQL

We will re-review the SPARQL documents, in particular the Query Language:

* <https://www.w3.org/TR/sparql12-query/>

We will cover:

1. The multiple SPARQL specifications
2. Property Paths
3. Assignment
4. Aggregation, ORDER BY & LIMIT
5. GRAPH
6. Functions
    * SPARQL 1.1 Functions
7. DESCRIBE
8. CONSTRUCT
9. INSERT
10. DELETE

## 3.1. The multiple SPARQL specifications

All the SPARQL specifications are linked to in the [Set of Documents](https://www.w3.org/TR/sparql12-query/#related) section in each which lists

1. What’s New in SPARQL 1.2 - _not ready yet!_
2. SPARQL 1.2 Concepts
3. SPARQL 1.2 Query Language
4. SPARQL 1.2 Update
5. SPARQL 1.2 Service Description
6. SPARQL 1.2 Federated Query
7. SPARQL 1.2 Query Results JSON Format
8. SPARQL 1.2 Query Results CSV and TSV Formats
9. SPARQL 1.2 Query Results XML Format
10. SPARQL 1.2 Entailment Regimes
11. SPARQL 1.2 Protocol
12. SPARQL 1.2 Graph Store Protocol

These are all spelled out in [SPARQL 1.2 Concepts](https://w3c.github.io/sparql-concepts/spec/)

We will only need to cover the following for now:

* Query Language
* Query Results JSON Format

Next lesson we will cover:

* Update

You have already seen Protocol in action: "a means for conveying SPARQL queries and updates to a SPARQL processing service and returning the results via HTTP".

There is the related Graph Store Protocol - "the use of HTTP operations for the purpose of managing a collection of RDF graphs" but we won't need taht just yet.


## 3.2. Property Paths

```mermaid
flowchart LR
    d["Dataset A"]
    px["Person X"]
    d --prov:qualifiedAttribution--> qa1
    qa1 --prov:agent--> px
    qa1 --prov:hadRole--> dr:custodian
```


There are two property paths here:

```mermaid
flowchart LR
    d --prov:qualifiedAttribution / prov:agent--> px
```

and

```mermaid
flowchart LR
    d --prov:qualifiedAttribution / prov:hadRole--> dr:custodian
```

and remember the person's name is a further step:

```mermaid
flowchart LR
    d["Dataset A"]
    px["Person X"]
    d --prov:qualifiedAttribution / prov:agent / schema:name--> px
```

In [ ]:
from IPython.display import display, Markdown
from kurra.sparql import query
from kurra.utils import render_sparql_result

def table_print(r):
    display(Markdown(render_sparql_result(r)))

rdf_bn = """
PREFIX dr: <https://linked.data.gov.au/def/data-roles/>
PREFIX ex: <http://example.com/>
PREFIX people: <https://linked.data.gov.au/dataset/people/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX schema: <https://schema.org/>

ex:a
    a schema:Dataset ;
    schema:name "Dataset A" ;
    prov:qualifiedAttribution [
        prov:agent people:px ;
        prov:hadRole dr:custodian ;
    ] ,
    [
        prov:agent people:py ;
        prov:hadRole dr:rightsHolder
    ] ;
.

people:px
    a schema:Person ;
    schema:name "Person X" ;
.

people:py
    a schema:Person ;
    schema:name "Person Y" ;
.
"""

For the data above, alter the query below to select:

1. Person X's name and Person Y's name
2. Person X's name
3. Dataste A's name, starting from Person Y
4. Person X's name from Person Y
5. Person X's name from Person Y, without filtering by role

In [ ]:
q = """
PREFIX dr: <https://linked.data.gov.au/def/data-roles/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX schema: <https://schema.org/>

SELECT ?name
WHERE {       
    # ex:a ALTER-THIS ?name .  
}
"""

In [ ]:
r = query(rdf_bn, q)
table_print(r)

### 3.2.1 More Path expressions elements

As per <https://www.w3.org/TR/sparql12-query/#pp-language>.


#### Sequence & Inverse

Operator | Name | Description
--- | --- | ---
`/` | Sequence | `x/y/z` is `x` then `y` then `z`
`^` | Inverse | reverse direction

As per the exercise above

#### Alternative & Negation

Operator | Name | Description
--- | --- | ---
`\|` | Alternative | `a\|b` is `a` or `b`
`!` | Negation | `!x` is anything but `x`

We will try these next

#### Path Operators

Operator | Name | Description
--- | --- | ---
`+` | One or more | Path of `x` or `x`/x` or `x/x/x...`
`*` | Zero or more | Path of None, `x` or `x`/x` or `x/x/x...`
`?` | One or Zero | None or `x`

Also next

In [ ]:
# data for Alternative & Negation demos
rdf_data = \
    """
    PREFIX ex: <http://example.com/>
    PREFIX schema: <https://schema.org/>
    
    ex:nick 
        a schema:Person ;
        ex:likes ex:icecream ;
        ex:loves ex:liquorice ;
        ex:hates ex:eggplant ;
    .    
    """

# Alternate
q = """
    SELECT *
    WHERE {
        ?nick ex:likes|ex:loves ?food
    }
    """

r = query(rdf_data, q)
table_print(r)

In [ ]:
# Negation
q = """
    SELECT *
    WHERE {
        ?nick !ex:loves ?food
    }
    """

r = query(rdf_data, q)
table_print(r)

In [ ]:
# data for Path Operatos demos
rdf_data = \
    """
    PREFIX ex: <http://example.com/>
    PREFIX schema: <https://schema.org/>
    
    ex:nick 
        a schema:Person ;
        schema:parent ex:george ;
    .

    ex:george 
        a schema:Person ;
        schema:parent ex:miko ;
    .

    ex:miko 
        a schema:Person ;
        schema:parent ex:ivan ;
    .

    ex:ivan 
        a schema:Person ;
    .

    ex:mickie
        a schema:Person ;
        schema:parent ex:nick ;
    .    
    """

In [ ]:
# Parent of nick
q = """
    PREFIX ex: <http://example.com/>
    PREFIX schema: <https://schema.org/>

    SELECT ?p 
    WHERE {
        ex:nick schema:parent ?p .
    }
    """

r = query(rdf_data, q)
table_print(r)

In [ ]:
# All ancestors of nick
q = """
    PREFIX ex: <http://example.com/>
    PREFIX schema: <https://schema.org/>

    SELECT ?p 
    WHERE {
        ex:nick schema:parent+ ?p .
    }
    """

r = query(rdf_data, q)
table_print(r)

In [ ]:
# nick and all his ancestors 
q = """
    PREFIX schema: <https://schema.org/>

    SELECT ?p 
    WHERE {
        ex:nick schema:parent* ?p
    }
    """

r = query(rdf_data, q)
table_print(r)

In [ ]:
# nick, his parent and grandparent
q = """
    PREFIX schema: <https://schema.org/>

    SELECT ?p 
    WHERE {
        ex:nick schema:parent/schema:parent? ?p
    }
    """

r = query(rdf_data, q)
table_print(r)

There are more... but that's enough! See <https://www.w3.org/TR/sparql12-query/#pp-language>

## 3.3. Assignment

Assigning values to variables in queries.

* BIND
* VALUES

### BIND

Assigning a static value or the result of a calculation to a variable.

For static values, see the **Sequence & Inverse section**, above.

For results of calculation:

In [ ]:
from IPython.display import display, Markdown
from kurra.sparql import query
from kurra.utils import render_sparql_result


def table_print(r):
    display(Markdown(render_sparql_result(r)))
    

rdf = """
PREFIX people: <https://linked.data.gov.au/dataset/people/>
PREFIX schema: <https://schema.org/>

people:nick
    a schema:Person ;
    schema:name "Nick" ;
    schema:age 42 ;
.
"""

q = """
PREFIX schema: <https://schema.org/>

SELECT ?age ?ageInMonths
WHERE {
    ?p schema:age ?age .

    BIND ((?age*12) AS ?ageInMonths)
}
"""

r = query(rdf, q)
table_print(r)

### VALUES

Assigning multiple values (static or as a result of calculations) to a variable.

Literals example:

In [ ]:
from IPython.display import display, Markdown
from kurra.sparql import query
from kurra.utils import render_sparql_result


def table_print(r):
    display(Markdown(render_sparql_result(r)))
    

rdf = """
PREFIX people: <https://linked.data.gov.au/dataset/people/>
PREFIX schema: <https://schema.org/>

people:nick
    a schema:Person ;
    schema:name "Nick" ;
    schema:age 42 ;
.

people:george
    a schema:Person ;
    schema:name "George" ;
    schema:age 70 ;
.

people:cathy
    a schema:Person ;
    schema:name "Cathy" ;
    schema:age 68 ;
.
"""

q = """
PREFIX people: <https://linked.data.gov.au/dataset/people/>
PREFIX schema: <https://schema.org/>

SELECT ?p
WHERE {
    VALUES ?p {
        people:nick
        people:bob  # does not exist
    }
    
    ?p schema:name ?name .
}
"""

r = query(rdf, q)
table_print(r)

In [ ]:
# binding with literals
q = """
PREFIX schema: <https://schema.org/>

SELECT ?p
WHERE {
    VALUES ?name {
        "Nick"
        "Bob"  # does not exist
    }
    
    ?p schema:name ?name .
}
"""

r = query(rdf, q)
table_print(r)

For an IRIs example, see the **Alternative & Negation section**, above.

## 3.4. Aggregation, ORDER BY & LIMIT

Just like SQL...

In [ ]:
from IPython.display import display, Markdown
from kurra.sparql import query
from kurra.utils import render_sparql_result

def table_print(r):
    display(Markdown(render_sparql_result(r)))
    

rdf = """
PREFIX people: <https://linked.data.gov.au/dataset/people/>
PREFIX schema: <https://schema.org/>

people:nick
    a schema:Person ;
    schema:name "Nick" ;
    schema:age 42 ;
.

people:george
    a schema:Person ;
    schema:name "George" ;
    schema:age 70 ;
.

people:cathy
    a schema:Person ;
    schema:name "Cathy" ;
    schema:age 68 ;
.
"""

q = """
PREFIX schema: <https://schema.org/>

SELECT ?p
WHERE {
    ?p 
        a schema:Person ;
        schema:age ?age ;
    .
}
ORDER BY DESC(?age)
LIMIT 2
"""

r = query(rdf, q)
table_print(r)

## 3.5. GRAPH

Selecting data from only a portion of a database - one graph amongst many.

This is somewhat analogous to SQL's `FROM` to get data from a single table.

Most modern RDF DBs store multiple **GRAPH**s of data:

&nbsp; | &nbsp; | &nbsp; | &nbsp; 
--- | --- | --- | --- 
`subject` | `predicate` | `object` | `graph`

`graph` then is just another filter/subset...

```sparql
SELECT * 
WHERE {
    GRAPH ?g {
        ?s ?p ?o
    }
}
```

> **NOTE**: RDF DBs are _sometimes_ configured to search all GRAPHs if none are specified but not always... so if you get no results, try with and without GRAPH

In [ ]:
rdf_data = \
"""
PREFIX ex: <http://example.com/>
PREFIX people: <https://linked.data.gov.au/dataset/people/>
PREFIX schema: <https://schema.org/>

ex:g1 {
    people:nick
        a schema:Person ;
        schema:name "Nick" ;
        schema:age 42 ;
    .
}

<http://anotherexample.com/g2> {
    people:george
        a schema:Person ;
        schema:name "George" ;
        schema:age 70 ;
    .
    
    people:cathy
        a schema:Person ;
        schema:name "Cathy" ;
        schema:age 68 ;
    .
}
"""

from rdflib import Dataset

d = Dataset().parse(data=rdf_data, format="trig")

# how many triples in the Dataset?
print(len(d))

In [ ]:
# how many triples in graph `ex:g1`?
print(len(d.graph("http://example.com/g1")))

In [ ]:
# how many triples in graph `http://anotherexample.com/g2`?
print(len(d.graph("http://anotherexample.com/g2")))

In [ ]:
# naivly selecting for all triples gets nothing...
q = """
    SELECT ?p 
    WHERE {
        ?p a schema:Person .
    }
    """

c = 0
for r in d.query(q):
    print(r)
    c += 1
print(f"Count: {c}")

In [ ]:
# check for triples in Named Graphs
q = """
    SELECT ?p 
    WHERE {
        GRAPH ?g {
            ?p a schema:Person .
        }
    }
    """

for r in d.query(q):
    print(str(r["p"]))

In [ ]:
# change the Dataset to union graph then re-run the naive query
d.default_union = True

q = """
    SELECT ?p
    WHERE {
        ?p a schema:Person .
    }
    """

c = 0
for r in d.query(q):
    print(r["p"])
    c += 1
print(f"Count: {c}")

### Interactive Exercise

Now we will to the same 2-graph exercise above using GraphDB.

## 3.6. Functions

SPARQL [defines](https://www.w3.org/TR/sparql12-query/#SparqlOps) a long list of in-built functions.

Here are a few function examples:

In [ ]:
# generate a UUID URN
rdf = "PREFIX : <http://example.com/> :a :b :c ."
q = """
    SELECT (UUID() AS ?uuid)
    WHERE {}
    """
r = query(rdf, q)
table_print(r)

In [ ]:
# cast it to a string literal with STR()
rdf = "PREFIX : <http://example.com/> :a :b :c ."
q = """
    SELECT (STR(UUID()) AS ?uuid)
    WHERE {}
    """
r = query(rdf, q)
table_print(r)

In [ ]:
# remove the "urn:uuid:" bit
rdf = "PREFIX : <http://example.com/> :a :b :c ."
q = """
    SELECT (STRAFTER(STR(UUID()), "uuid:") AS ?uuid)
    WHERE {}
    """
r = query(rdf, q)
table_print(r)

In [ ]:
# get the IRI and the string form using BIND
rdf = "PREFIX : <http://example.com/> :a :b :c ."
q = """
    SELECT *
    WHERE {
      BIND (UUID() AS ?uuid)
      BIND (STRAFTER(STR(?uuid), "uuid:") AS ?str_uuid)
    }
    """
r = query(rdf, q)
table_print(r)

### Literal filters

`CONTAINS(...)` will check to see if a literal contains a substring:

`CONTAINS("Nicholas", "Nic")` --> True  
`CONTAINS("Nicholas", "Bob")` --> False

SPARQL functions return Tue or False and, if True, that part of the SPARQL function returns all values.

In [ ]:
rdf = """
PREFIX people: <https://linked.data.gov.au/dataset/people/>
PREFIX schema: <https://schema.org/>

people:nick
    a schema:Person ;
    schema:name "Nick" ;
    schema:age 42 ;
.

people:george
    a schema:Person ;
    schema:name "George" ;
    schema:age 70 ;
.

people:cathy
    a schema:Person ;
    schema:name "Cathy" ;
    schema:age 68 ;
.
"""

# a filter with a boolean operator
q = """
    PREFIX schema: <https://schema.org/>
    
    SELECT *
    WHERE {
        ?p 
            a schema:Person ;
            schema:name ?name ;
        .
    
        FILTER (CONTAINS(?name, "c") || CONTAINS(?name, "C"))
    
    }
    """

r = query(rdf, q)
table_print(r)

In [ ]:
# REGEX() - I use this one a lot
q = """
    PREFIX schema: <https://schema.org/>
    
    SELECT *
    WHERE {
        ?p 
            a schema:Person ;
            schema:name ?name ;
        .
    
        FILTER (REGEX(?name, "c", "i"))  # try with "^c"
    
    }
    """

r = query(rdf, q)
table_print(r)

> WARNING: The order of applying filters like CONTAINS is really important!
>
> If we have a DB of 1M people, we do _NOT_ want to query like above as al 1M named will be tested for a match.

Instead, we want to restict the results from a Graph Pattern Match as much as possible first.

> NOTE: we will look at text indexes in Module 4

## 3.7. DESCRIBE

Simple but powerful!

Gets all the inbound and outbound edges & nodes for a selected node.

In [ ]:
rdf = """
PREFIX people: <https://linked.data.gov.au/dataset/people/>
PREFIX schema: <https://schema.org/>

people:nick
    a schema:Person ;
    schema:name "Nick" ;
    schema:age 42 ;
.

people:george
    a schema:Person ;
    schema:name "George" ;
    schema:age 70 ;
.

people:cathy
    a schema:Person ;
    schema:name "Cathy" ;
    schema:age 68 ;
.
"""

# target a single node
q = """DESCRIBE <https://linked.data.gov.au/dataset/people/nick>"""

r = query(rdf, q)
table_print(r)

In [ ]:
# find the target node with WHERE - can be more than one node, returns a graph
q = """
    PREFIX schema: <https://schema.org/>
    DESCRIBE ?p
    WHERE {
        ?p 
            a schema:Person ;
            schema:name ?name ;
        .

        # remove the FILTER too
        FILTER(CONTAINS(?name, "N"))
    }
    """

r = query(rdf, q)
table_print(r)

## 3.8. CONSTRUCT

`CONSTRUCT` lets you make a graph return, rather than a table.

`CONSTRUCT` is frequently used to return an RDF subgraph of a larger graph. APIs often use it, people, less so.

In [ ]:
# a pretty-print function
def construct_print(rdf, query):
    from rdflib import Graph
    g = Graph(bind_namespaces="none").parse(data=rdf_data, format="turtle")
    x = g.query(q)
    for prefix, namespace in g.namespaces():
        x.graph.bind(prefix, namespace)

    print(x.serialize(format="turtle").decode())

In [ ]:
rdf = """
PREFIX people: <https://linked.data.gov.au/dataset/people/>
PREFIX schema: <https://schema.org/>

people:nick
    a schema:Person ;
    schema:name "Nick" ;
    schema:age 42 ;
    schema:parent
        people:george ,
        people:cathy ;
.

people:george
    a schema:Person ;
    schema:name "George" ;
    schema:age 70 ;
    schema:parent people:miko ;
.

people:cathy
    a schema:Person ;
    schema:name "Cathy" ;
    schema:age 68 ;
.

people:miko
    a schema:Person ;
    schema:name "Miko" ;    
.
"""


# just get all the people, in graph form
q = """
    PREFIX schema: <https://schema.org/>

    CONSTRUCT {
        ?p 
            a schema:Patient ;
            schema:firstName ?name
        .
    }
    WHERE {
        ?p 
            a schema:Person ;
            schema:name ?name
    }
    
    """

r = query(rdf, q)
table_print(r)

In [ ]:
# Create a new predicate: grandparent = parent + parent
# when the commented out values are used
q = """
    PREFIX ex: <http://example.com/>
    PREFIX schema: <https://schema.org/>
    
    CONSTRUCT {
        ?p1 schema:parent ?p2
        # ?p1 ex:grandParent ?p2
    }
    WHERE {
        ?p1 
            schema:parent/schema:parent ?p2 ;
        .
        # ?p1 
        #    schema:parent/schema:parent ?p2 ;
        # .
    }
    """

# construct_print(rdf_data, q)

r = query(rdf, q)
table_print(r)

## 3.9 INSERT

We will do some INSERTing in GraphDB...

1. INSERT new data
2. INSERT WHERE
3. INSERT from one graph to another

For 1.:
```sparql
PREFIX schema: <https://schema.org/>
PREFIX people: <https://linked.data.gov.au/dataset/people/>

INSERT DATA {
    people:bob 
        a schema:Person ;
    	schema:name "Bob" ;
    .
}
```

then

```sparql
PREFIX people: <https://linked.data.gov.au/dataset/people/>
DESCRIBE people:bob
```

For 2.:
```sparql
PREFIX schema: <https://schema.org/>

INSERT {
    ?p
    	schema:age 101 ;
    .
}
WHERE {
	?p
        a schema:Person ;
	    schema:name "Bob"
   .
}
```

then

```sparql
PREFIX people: <https://linked.data.gov.au/dataset/people/>
DESCRIBE people:bob
```

For 3.:
```sparql
PREFIX schema: <https://schema.org/>

INSERT {
    GRAPH <http://thing.com/g> {
	    ?p
    		schema:age 102 ;
    	.
    }
}
WHERE {
	?p
        a schema:Person ;
	    schema:name "Bob"
   .
}
```

then

```sparql
PREFIX people: <https://linked.data.gov.au/dataset/people/>
SELECT *
WHERE {
    GRAPH ?g {
    	people:bob ?p ?o
    }
}
```

Note that this only shows the results from Named Graphs, not the default graph

## 3.10 DELETE

Following on from above:

4. DELETE WHERE
5. DELETE FILTER
6. DELETE + INSERT WHERE


For 4.:
```sparql
PREFIX schema: <https://schema.org/>

DELETE {
    ?s a schema:Person
}
WHERE {
    ?s ?p ?o
}
```

For 5.:
```sparql
PREFIX schema: <https://schema.org/>

DELETE {
    ?s schema:name ?n
}
WHERE {
    ?s schema:name ?n
    
    FILTER (CONTAINS(?n, "N"))
}
```

For 6.:

```sparql
PREFIX people: <https://linked.data.gov.au/dataset/people/>
PREFIX schema: <https://schema.org/>

SELECT *
WHERE {
    people:nick schema:name ?name
}
```

then

```sparql
PREFIX people: <https://linked.data.gov.au/dataset/people/>
PREFIX schema: <https://schema.org/>

DELETE {
    people:nick schema:name ?n
}
INSERT {
    people:nick schema:name "Mikołaj"
}
WHERE {
    people:nick schema:name ?n
}
```

then

```sparql
PREFIX people: <https://linked.data.gov.au/dataset/people/>
PREFIX schema: <https://schema.org/>

SELECT *
WHERE {
    people:nick schema:name ?name
}
```